<a href="https://colab.research.google.com/github/Jubicod/wsf/blob/main/tutorial1/install.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt install gcc-arm-none-eabi

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  binutils-arm-none-eabi libnewlib-arm-none-eabi libnewlib-dev libstdc++-arm-none-eabi-dev
  libstdc++-arm-none-eabi-newlib
Suggested packages:
  libnewlib-doc
The following NEW packages will be installed:
  binutils-arm-none-eabi gcc-arm-none-eabi libnewlib-arm-none-eabi libnewlib-dev
  libstdc++-arm-none-eabi-dev libstdc++-arm-none-eabi-newlib
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 442 MB of archives.
After this operation, 2,575 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 binutils-arm-none-eabi amd64 2.38-3ubuntu1+15build1 [3,040 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 gcc-arm-none-eabi amd64 15:10.3-2021.07-4 [47.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libstdc++-arm-none-eabi-dev all 15:10.3-2

In [2]:
!git clone https://github.com/Jubicod/wsf.git

Cloning into 'wsf'...
remote: Enumerating objects: 155, done.
remote: Counting objects: 100% (155/155), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 155 (delta 60), reused 146 (delta 55), pack-reused 0
Receiving objects: 100% (155/155), 153.94 KiB | 1.44 MiB/s, done.
Resolving deltas: 100% (60/60), done.


In [3]:
!pip install unicorn
!pip install lief

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 6.5 MB/s eta 0:00:00


In [4]:
!make -C wsf/tutorial1/app clean all

make: Entering directory '/content/wsf/tutorial1/app'
rm -f app.elf
rm -f app.txt
rm -f app.map
arm-none-eabi-gcc -O1 -g -march=armv7-m startup_stm32f10x_ld.s main.c slots.c --specs=nosys.specs -T stm32_flash.ld -o app.elf
arm-none-eabi-objdump -S app.elf > app.txt
arm-none-eabi-objdump -x app.elf > app.map
make: Leaving directory '/content/wsf/tutorial1/app'


In [5]:
import unicorn as uc
import lief
import weakref

class Printf:

  def __init__(self, emu):
    self.emu = emu
    self.param = 0
    self.sp = self.emu.reg_read(uc.arm_const.UC_ARM_REG_SP)
    r0 = self.emu.reg_read(uc.arm_const.UC_ARM_REG_R0)
    format = self.get_string(r0)
    i = 0
    while i < len(format):
      if format[i]=='%' and i < (len(format)-1):
        i+=1
        t = format[i]
        n = self.next_param()
        if t=='s':
          print(self.get_string(n), end='')
        elif t=='c':
          print(chr(n & 0xff), end='')
        elif t=='d':
          print(str(n), end='')
        elif t=='x' or t=='p':
          print(hex(n), end='')
        else:
          print(format[i-1:i+1], end='')
      else:
        print(format[i], end='')
      i+=1

  def next_param(self):
    if self.param == 0:
      p = self.emu.reg_read(uc.arm_const.UC_ARM_REG_R1)
    elif self.param == 1:
      p = self.emu.reg_read(uc.arm_const.UC_ARM_REG_R2)
    elif self.param == 2:
      p = self.emu.reg_read(uc.arm_const.UC_ARM_REG_R3)
    else:
      p = int.from_bytes(self.emu.mem_read(self.sp+(self.param-3)*4, 4), "little")
    self.param += 1
    return p

  def get_string(self, address):
    s = ''
    while(1):
      c = self.emu.mem_read(address,1)[0]
      if c==0:
        break
      address += 1
      s += chr(c)
    return s

class HookWeakMethod:
    """
    Class to pass instance method callbacks to unicorn with weak referencing to
    prevent circular dependencies.

    Circular dependencies blocks the GC to clean the rainbowBase at the correct
    time, and this causes memory troubles...

    We cannot use directly weakref.WeakMethod since __call__ does not execute
    the method, but returns it. This class does call the method when __call__
    is executed.
    """
    def __init__(self, method):
        self.method = weakref.WeakMethod(method)

    def __call__(self, *args, **kwargs):
        self.method()(*args, **kwargs)

class App:

  def __init__(self, elf_file='wsf/tutorial1/app/app.elf'):
    self.functions = {}
    self.function_names = {}
    self.ins_counter = 0
    self.elf_file = elf_file
    self.emu = uc.Uc(uc.UC_ARCH_ARM, uc.UC_MODE_THUMB | uc.UC_MODE_MCLASS)
    self.emu.mem_map(0x00000000, 0x1000, uc.UC_PROT_ALL) # ugly hack, printf issue ?
    self.emu.mem_map(0x20000000, 0x10000, uc.UC_PROT_ALL) # make RAM executable for now
    self.emu.mem_map(0x08000000, 0x10000, uc.UC_PROT_ALL) # make FLASH writable for now
    self.load()
    self.emu.mem_protect(0x08000000, 0x10000, uc.UC_PROT_READ | uc.UC_PROT_EXEC)  # make flash non writable
    #self.emu.mem_protect(0x20000000, 0x10000, uc.UC_PROT_READ | uc.UC_PROT_WRITE)  # make RAM not executable
    self.set_hook('puts', HookWeakMethod(self.hook_puts))
    self.set_hook('putchar', HookWeakMethod(self.hook_putchar))
    self.set_hook('getchar', HookWeakMethod(self.hook_getchar))
    self.set_hook('printf', HookWeakMethod(self.hook_printf))
    #self.set_hook('__sfputc_r', HookWeakMethod(self.hook_))
    #self.emu.hook_add(uc.UC_HOOK_MEM_WRITE, HookWeakMethod(self.hook_write_mem))
    #self.emu.hook_add(uc.UC_HOOK_CODE, HookWeakMethod(self.hook_code), begin=self.functions['receive_command']-1, end=0x08000278)
    self.emu.hook_add(uc.UC_HOOK_CODE, HookWeakMethod(self.hook_code_count), begin=0x08000000, end=0x08010000)
    self.reset()

  def set_hook(self, function, hook):
    add = self.functions[function]
    self.emu.hook_add(uc.UC_HOOK_CODE, hook, begin=add-1, end=add)

  def load(self, verbose=False):
    """ Load an .elf file into emu's memory using LIEF """
    elffile = lief.parse(self.elf_file)
    if verbose:
        print(f"[x] Loading .elf ...")

    if len(list(elffile.segments)) > 0:
        for segment in elffile.segments:
            if segment.type == lief.ELF.SEGMENT_TYPES.LOAD:
                for section in segment.sections:
                    if verbose:
                        print(
                            f"[=] Writing {section.name} on {section.virtual_address:x} - {section.virtual_address+section.size:x}"
                        )
                    self.emu.mem_write(section.virtual_address, bytes(section.content))

    # lief > 0.10
    try:
        for f in elffile.exported_functions:
            tmpn = f.name
            c = 0
            while tmpn in self.functions:
                c += 1
                tmpn = f.name + str(c)
            self.functions[tmpn] = f.address
    except:
        pass

    ## TODO: when the ELF has relocated functions exported, LIEF fails on get_function_address
    for i in elffile.symbols:
        if i.type == lief.ELF.SYMBOL_TYPES.FUNC:
            try:
                tmpn = i.name
                addr = i.value
                if self.functions[tmpn] != addr:
                    c = 0
                    while tmpn in self.functions.keys():
                        c += 1
                        tmpn = i.name + str(c)
                    self.functions[tmpn] = addr
            except Exception as e:
                if verbose:
                    print(e, i)

    self.function_names = {self.functions[x]: x for x in self.functions.keys()}


  def read32(self, address):
    return int.from_bytes(self.emu.mem_read(address, 4), "little")

  def reset(self):
    self.ins_counter = 0
    self.emu.reg_write(uc.arm_const.UC_ARM_REG_SP, self.read32(0x08000000))
    self.emu.reg_write(uc.arm_const.UC_ARM_REG_PC, self.read32(0x08000004))

  def hook_puts(self, emu, address, size, user_data):
    r0 = emu.reg_read(uc.arm_const.UC_ARM_REG_R0)
    while(1):
      c = emu.mem_read(r0,1)[0]
      if c==0:
        break
      r0 += 1
      print(chr(c), end='')
    print('')
    self.ret()

  def hook_putchar(self, emu, address, size, user_data):
    r0 = emu.reg_read(uc.arm_const.UC_ARM_REG_R0)
    print(chr(r0 & 0xff), end='')
    self.ret()


  def hook_write(self, emu, address, size, user_data):
    #r0 = emu.reg_read(uc.arm_const.UC_ARM_REG_R0)
    #while(1):
    #  c = emu.mem_read(r0,1)[0]
    #  if c==0:
    #    break
    #  r0 += 1
    #  print(chr(c), end='')
    #print('')
    print('WRITE')
    self.ret()

  def hook_getchar(self, emu, address, size, user_data):
    if self.cmd_idx > len(self.cmd_buf):
      raise NameError('Finished')
    if self.cmd_idx < len(self.cmd_buf):
      c = self.cmd_buf[self.cmd_idx]
    else:
      c = ' ' #last one must be a space
    emu.reg_write(uc.arm_const.UC_ARM_REG_R0, ord(c))
    self.cmd_idx += 1
    self.ret()

  def hook_printf(self, emu, address, size, user_data):
    Printf(emu)
    self.ret()


  def hook_code(self, emu, address, size, user_data):
    print(">>> Tracing instruction at 0x%x, instruction size = 0x%x" %(address, size))

  def hook_code_count(self, emu, address, size, user_data):
      self.ins_counter += 1


  def hook_write_mem(self, emu, access, address, size, value, user_data):
    if access == uc.UC_MEM_WRITE:
      print(">>> Writing 0x%x at 0x%x" %(value, address))


  def ret(self):
    self.emu.reg_write(uc.arm_const.UC_ARM_REG_PC, self.emu.reg_read(uc.arm_const.UC_ARM_REG_LR))

  def send(self, command, timeout=10000):
    self.cmd_buf = command
    self.cmd_idx = 0
    self.ins_counter = 0
    try:
      self.emu.emu_start(self.emu.reg_read(uc.arm_const.UC_ARM_REG_PC)|1, 0x08000295)
    except NameError:
      pass
    except uc.UcError as e:
      print('------CRASHED--------')
      print(e)
      print('PC =', hex(self.emu.reg_read(uc.arm_const.UC_ARM_REG_PC)))
    return self.ins_counter

